In [1721]:
import pandas as pd
import numpy as np
import pickle
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

**Cleaning the IMDB dataframe**

Read pickle:

In [1722]:
imdb_data = pd.read_pickle('imdb_scraped_data.pkl')

In [1723]:
imdb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6864 entries, 0 to 6863
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie        6864 non-null   object 
 1   year         6864 non-null   object 
 2   imdb         6864 non-null   float64
 3   metascore    6864 non-null   object 
 4   votes        6864 non-null   int64  
 5   runtime      6864 non-null   object 
 6   certificate  6864 non-null   object 
 7   genre        6864 non-null   object 
 8   director     6864 non-null   object 
 9   stars        6864 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 536.4+ KB


Clean up the year feature:

In [1724]:
imdb_data.loc[:, 'year'] = imdb_data['year'].str[-5:-1]

In [1725]:
imdb_data.year = imdb_data.year.astype(int) #converted year to int

In [1726]:
imdb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6864 entries, 0 to 6863
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie        6864 non-null   object 
 1   year         6864 non-null   int64  
 2   imdb         6864 non-null   float64
 3   metascore    6864 non-null   object 
 4   votes        6864 non-null   int64  
 5   runtime      6864 non-null   object 
 6   certificate  6864 non-null   object 
 7   genre        6864 non-null   object 
 8   director     6864 non-null   object 
 9   stars        6864 non-null   object 
dtypes: float64(1), int64(2), object(7)
memory usage: 536.4+ KB


Drop records with missing metascore:

In [1727]:
imdb_data.metascore.value_counts()

no metascore    1152
64               129
65               124
66               122
53               119
                ... 
7                  2
99                 1
6                  1
5                  1
1                  1
Name: metascore, Length: 97, dtype: int64

In [1728]:
imdb_data = imdb_data[imdb_data.metascore != 'no metascore'] #drops 1152 of 6864 records (17%) 
imdb_data.metascore = imdb_data.metascore.astype(float)

Convert runtime to int:

In [1729]:
imdb_data.runtime = imdb_data.runtime.str.slice(0, -4) #stripped min from runtime
imdb_data.runtime = imdb_data.runtime.astype(int) #converted runtime to int

Exclude select ratings and group unrated ratings:

In [1730]:
imdb_data.certificate.value_counts()

R            2736
PG-13        1779
PG            655
Not Rated     309
G              95
TV-MA          60
not rated      27
Unrated        19
TV-14          16
NC-17          13
TV-PG           3
Name: certificate, dtype: int64

In [1731]:
imdb_data = imdb_data[imdb_data.certificate.isin(['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated', 'not rated', 'Unrated'])]                                
#excluded ratings of ['TV-14', 'TV-MA', 'TV-PG', 'TV-G', 'TV-Y7', 'TV-Y7-FV', 'A0', 'M', 'X', 'Approved']

In [1732]:
imdb_data['certificate'] = imdb_data['certificate'].str.replace('not rated','Not Rated')
imdb_data['certificate'] = imdb_data['certificate'] = imdb_data['certificate'].str.replace('Unrated','Not Rated')

In [1733]:
 imdb_data.certificate.value_counts()

R            2736
PG-13        1779
PG            655
Not Rated     355
G              95
NC-17          13
Name: certificate, dtype: int64

In [1734]:
imdb_data.sample(5)

,movie,year,imdb,metascore,votes,runtime,certificate,genre,director,stars
2271,Ray,2004,7.7,73.0,133087,152,PG-13,"[Biography,, Drama,, Music]",Taylor Hackford,"[Jamie Foxx, Regina King, Kerry Washington, Cl..."
3266,Code Name: The Cleaner,2007,4.6,33.0,11048,84,PG-13,"[Action,, Comedy,, Crime]",Les Mayfield,"[Cedric the Entertainer, Lucy Liu, Nicollette ..."
5675,Truth,2015,6.8,66.0,17815,125,R,"[Biography,, Drama,, History]",James Vanderbilt,"[Cate Blanchett, Robert Redford, Dennis Quaid,..."
970,Gummo,1997,6.8,19.0,30648,89,R,"[Comedy,, Drama]",Harmony Korine,"[Nick Sutton, Jacob Sewell, Lara Tosh, Jacob R..."
1906,Antwone Fisher,2002,7.3,62.0,31674,120,PG-13,"[Biography,, Drama]",Denzel Washington,"[Denzel Washington, Derek Luke, Joy Bryant, Ma..."


Add new feature as genre count:

In [1735]:
imdb_data['genre_count'] = imdb_data.genre.str.len() #number of genres added as new feature

Explode the genre and stars features into rows:

In [1736]:
imdb_data = imdb_data.explode('genre').reset_index(drop=True)
imdb_data = imdb_data.explode('stars').reset_index(drop=True)

Remove comma from genre:

In [1737]:
imdb_data['genre'] = imdb_data['genre'].str.replace(',', '')

In [1738]:
imdb_data.head(3)

,movie,year,imdb,metascore,votes,runtime,certificate,genre,director,stars,genre_count
0,Major League,1989,7.2,62.0,59421,107,R,Comedy,David S. Ward,Tom Berenger,2
1,Major League,1989,7.2,62.0,59421,107,R,Comedy,David S. Ward,Charlie Sheen,2
2,Major League,1989,7.2,62.0,59421,107,R,Comedy,David S. Ward,Corbin Bernsen,2


**Cleaning The Numbers dataframe**

Read pickle:

In [1739]:
thenumbers_data = pd.read_pickle('thenumbers_scraped_data.pkl')

In [1740]:
thenumbers_data.head()

,movie,release_date,production_budget,domestic_gross,worldwide_gross
0,Avengers: Endgame,"Apr 23, 2019","$400,000,000","$858,373,000","$2,797,800,564"
1,Pirates of the Caribbean: On Stranger Tides,"May 20, 2011","$379,000,000","$241,063,875","$1,045,663,875"
2,Avengers: Age of Ultron,"Apr 22, 2015","$365,000,000","$459,005,868","$1,403,013,963"
3,Star Wars Ep. VII: The Force Awakens,"Dec 16, 2015","$306,000,000","$936,662,225","$2,068,223,624"
4,Avengers: Infinity War,"Apr 25, 2018","$300,000,000","$678,815,482","$2,048,359,754"


In [1741]:
thenumbers_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6027 entries, 0 to 6026
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   movie              6027 non-null   object
 1   release_date       6027 non-null   object
 2   production_budget  6027 non-null   object
 3   domestic_gross     6027 non-null   object
 4   worldwide_gross    6027 non-null   object
dtypes: object(5)
memory usage: 235.6+ KB


In [1742]:
thenumbers_data.release_date[thenumbers_data.release_date.str.len() < 11].value_counts() #count records with incomplete release date (156)

Unknown       121
1993            4
1991            4
1980            3
1976            3
1978            3
1977            2
1987            2
Jun, 1979       1
1981            1
1975            1
Oct, 1993       1
1979            1
2012            1
1970            1
2008            1
1973            1
Mar, 1987       1
1971            1
1983            1
1968            1
Fall, 2021      1
Name: release_date, dtype: int64

Remove records with unknown or incomplete release dates:

In [1743]:
thenumbers_data = thenumbers_data[thenumbers_data.release_date.str.len() >= 11] #remove records with unknown release dates (121 rows)

In [1744]:
thenumbers_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5871 entries, 0 to 6026
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   movie              5871 non-null   object
 1   release_date       5871 non-null   object
 2   production_budget  5871 non-null   object
 3   domestic_gross     5871 non-null   object
 4   worldwide_gross    5871 non-null   object
dtypes: object(5)
memory usage: 275.2+ KB


In [1745]:
thenumbers_data['release_date'] = pd.to_datetime(thenumbers_data['release_date'], format='%b %d, %Y')

#thenumbers_dates = thenumbers_data.release_date.datetime()
#pd.to_datetime(thenumbers_data['release_date']).datetime.strftime('%b %d,%y')
#dateformatted = datetime.strptime('Apr 23, 2019', "%b %d, %Y").date()

In [1746]:
thenumbers_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5871 entries, 0 to 6026
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   movie              5871 non-null   object        
 1   release_date       5871 non-null   datetime64[ns]
 2   production_budget  5871 non-null   object        
 3   domestic_gross     5871 non-null   object        
 4   worldwide_gross    5871 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 275.2+ KB


Convert production budget, domestic gross and worldwide gross to numeric values:

In [1747]:
thenumbers_data['production_budget'] = thenumbers_data['production_budget'].str.replace(',', '').str.replace('$', '').astype(int)
thenumbers_data['domestic_gross'] = thenumbers_data['domestic_gross'].str.replace(',', '').str.replace('$', '').astype(int)
thenumbers_data['worldwide_gross'] = thenumbers_data['worldwide_gross'].str.replace(',', '').str.replace('$', '').astype(int)      

Add year column:

In [1748]:
thenumbers_data['year'] = thenumbers_data['release_date'].dt.year

In [1749]:
thenumbers_data.sample(5)

,movie,release_date,production_budget,domestic_gross,worldwide_gross,year
2084,Pitch Perfect 2,2015-05-07,29000000,184296230,287625468,2015
2849,Out of the Inferno,2016-03-08,19000000,0,0,2016
749,Underworld: Awakening,2012-01-20,70000000,62321039,160379930,2012
1209,Twisted,2004-02-27,50000000,25195050,40119848,2004
672,Inspector Gadget,1999-07-23,75000000,97387965,97387965,1999


**Cleaning the star dataframe**

Read pickle:

In [1750]:
star_data = pd.read_pickle('star_scraped_data.pkl')

In [1751]:
star_data.head()

,star_name,star_ranking,actor_or_actress
0,Julia Garner,1.,\n Actress
1,Tom Pelphrey,2.,\n Actor
2,Ana de Armas,3.,\n Actress
3,Eiza González,4.,\n Actress
4,Janet McTeer,5.,\n Actress


Remove period and space characters in star_ranking:

In [1752]:
star_data.star_ranking = star_data.star_ranking.str[:-2]

Remove \n and space characters in actor_or_actress:

In [1753]:
star_data.actor_or_actress = star_data.actor_or_actress.str.strip()

Add new feature called star_points which will be assigned as the reverse order of the star_ranking (e.g., star with the 1st star ranking will get 1000 points, 2nd star ranking 999 points, etc.)

In [1754]:
star_data['star_points'] = star_data.star_ranking.values[::-1]

Convert star_ranking and star_points to int:

In [1755]:
star_data['star_ranking'] = star_data.star_ranking.str.replace(',','').astype(int)
star_data['star_points'] = star_data.star_points.str.replace(',','').astype(int)

In [1756]:
star_data

,star_name,star_ranking,actor_or_actress,star_points
0,Julia Garner,1,Actress,1000
1,Tom Pelphrey,2,Actor,999
2,Ana de Armas,3,Actress,998
3,Eiza González,4,Actress,997
4,Janet McTeer,5,Actress,996
...,...,...,...,...
995,Halle Berry,996,Actress,5
996,Cynthia Erivo,997,Actress,4
997,Maura Tierney,998,Actress,3
998,Jennifer Carpenter,999,Actress,2


# Merge IMDB and The Numbers data:

In [1757]:
merged_df = pd.merge(imdb_data, thenumbers_data, how='left', left_on=['movie', 'year'], right_on=['movie', 'year'])

In [1758]:
merged_df.shape

(59591, 15)

In [1759]:
merged_df.sample(3)

,movie,year,imdb,metascore,votes,runtime,certificate,genre,director,stars,genre_count,release_date,production_budget,domestic_gross,worldwide_gross
34294,Cop Out,2010,5.6,31.0,83676,107,R,Crime,Kevin Smith,Juan Carlos Hernández,3,2010-02-26,37000000.0,44875481.0,55909910.0
18344,Dodgeball,2004,6.7,55.0,221542,92,PG-13,Comedy,Rawson Marshall Thurber,Ben Stiller,2,NaT,NaN,NaN,NaN
5007,A Little Princess,1995,7.7,83.0,30764,97,G,Drama,Alfonso Cuarón,Liam Cunningham,3,NaT,NaN,NaN,NaN


Exclude films without release dates:

In [1760]:
merged_df.dropna(subset=['release_date'], inplace=True)

Exclude films released in 2020 (or yet to be released):

In [1761]:
merged_df = merged_df[merged_df.release_date < 'Jan 1, 2020'] 

Merge with stars data:

In [1762]:
merged_df = pd.merge(merged_df, star_data, how='left', left_on=['stars'], right_on=['star_name'])

In [1763]:
merged_df.head(3)

,movie,year,imdb,metascore,votes,runtime,certificate,genre,director,stars,genre_count,release_date,production_budget,domestic_gross,worldwide_gross,star_name,star_ranking,actor_or_actress,star_points
0,Major League,1989,7.2,62.0,59421,107,R,Comedy,David S. Ward,Tom Berenger,2,1989-04-07,11000000.0,49793054.0,49793054.0,NaN,NaN,NaN,NaN
1,Major League,1989,7.2,62.0,59421,107,R,Comedy,David S. Ward,Charlie Sheen,2,1989-04-07,11000000.0,49793054.0,49793054.0,NaN,NaN,NaN,NaN
2,Major League,1989,7.2,62.0,59421,107,R,Comedy,David S. Ward,Corbin Bernsen,2,1989-04-07,11000000.0,49793054.0,49793054.0,NaN,NaN,NaN,NaN


If actor / actress not in top 1000 list, then give 0 star points:

In [1764]:
merged_df['star_points'].fillna(0, inplace=True)

In [1766]:
merged_df.head()

,movie,year,imdb,metascore,votes,runtime,certificate,genre,director,stars,genre_count,release_date,production_budget,domestic_gross,worldwide_gross,star_name,star_ranking,actor_or_actress,star_points
0,Major League,1989,7.2,62.0,59421,107,R,Comedy,David S. Ward,Tom Berenger,2,1989-04-07,11000000.0,49793054.0,49793054.0,NaN,NaN,NaN,0.0
1,Major League,1989,7.2,62.0,59421,107,R,Comedy,David S. Ward,Charlie Sheen,2,1989-04-07,11000000.0,49793054.0,49793054.0,NaN,NaN,NaN,0.0
2,Major League,1989,7.2,62.0,59421,107,R,Comedy,David S. Ward,Corbin Bernsen,2,1989-04-07,11000000.0,49793054.0,49793054.0,NaN,NaN,NaN,0.0
3,Major League,1989,7.2,62.0,59421,107,R,Comedy,David S. Ward,Margaret Whitton,2,1989-04-07,11000000.0,49793054.0,49793054.0,NaN,NaN,NaN,0.0
4,Major League,1989,7.2,62.0,59421,107,R,Sport,David S. Ward,Tom Berenger,2,1989-04-07,11000000.0,49793054.0,49793054.0,NaN,NaN,NaN,0.0


# Create new features called "Star Power", which is the average domestic gross that the star performed in prior movies, and "Star Appearances", which  is the # of previous movies that the star performed in:

Star Power feature:

In [1774]:
merged_df_reduced = merged_df[['movie','release_date','stars','domestic_gross']].drop_duplicates()
merged_df_reduced['star_power'] = merged_df_reduced.apply(lambda x: merged_df_reduced[(merged_df_reduced['stars'] == x['stars'])&(merged_df_reduced['release_date'] < x['release_date'])].domestic_gross.mean(), axis=1)
merged_df_reduced['star_appearances'] = merged_df_reduced.apply(lambda x: merged_df_reduced[(merged_df_reduced['stars'] == x['stars'])&(merged_df_reduced['release_date'] < x['release_date'])].shape[0], axis=1)
merged_df = merged_df.merge(merged_df_reduced2)

In [1775]:
merged_df['star_power'].fillna(0, inplace=True)

In [1776]:
merged_df.sample(5)

,movie,year,imdb,metascore,votes,runtime,certificate,genre,director,stars,...,release_date,production_budget,domestic_gross,worldwide_gross,star_name,star_ranking,actor_or_actress,star_points,star_appearances,star_power
33302,The Angry Birds Movie 2,2019,6.4,60.0,16069,97,PG,Adventure,Thurop Van Orman,Leslie Jones,...,2019-08-02,65000000.0,41657076.0,150460970.0,NaN,NaN,NaN,0.0,1,1.283506e+08
11965,Veer-Zaara,2004,7.8,67.0,46722,192,Not Rated,Musical,Yash Chopra,Preity Zinta,...,2004-11-12,7000000.0,2938532.0,7017859.0,NaN,NaN,NaN,0.0,0,0.000000e+00
15779,Michael Clayton,2007,7.2,82.0,151981,119,R,Crime,Tony Gilroy,Tom Wilkinson,...,2007-10-05,21500000.0,49033882.0,92987651.0,NaN,NaN,NaN,0.0,9,5.787503e+07
11083,50 First Dates,2004,6.8,48.0,314792,99,PG-13,Comedy,Peter Segal,Sean Astin,...,2004-02-13,75000000.0,120776832.0,196320329.0,Sean Astin,916.0,Actor,85.0,0,0.000000e+00
29828,Loving,2016,7.0,79.0,29701,123,PG-13,Drama,Jeff Nichols,Joel Edgerton,...,2016-11-04,9000000.0,7710234.0,12898064.0,NaN,NaN,NaN,0.0,11,4.551993e+07


In [1777]:
merged_df[merged_df.stars=='Jack Nicholson'].sort_values(by='release_date', ascending=False)

,movie,year,imdb,metascore,votes,runtime,certificate,genre,director,stars,...,release_date,production_budget,domestic_gross,worldwide_gross,star_name,star_ranking,actor_or_actress,star_points,star_appearances,star_power
15705,The Bucket List,2007,7.4,42.0,222556,97,PG-13,Drama,Rob Reiner,Jack Nicholson,...,2007-12-25,45000000.0,93466502.0,174807445.0,Jack Nicholson,359.0,Actor,642.0,9,1.034468e+08
15704,The Bucket List,2007,7.4,42.0,222556,97,PG-13,Comedy,Rob Reiner,Jack Nicholson,...,2007-12-25,45000000.0,93466502.0,174807445.0,Jack Nicholson,359.0,Actor,642.0,9,1.034468e+08
13583,The Departed,2006,8.5,85.0,1132417,151,R,Thriller,Martin Scorsese,Jack Nicholson,...,2006-10-06,90000000.0,132384315.0,289660619.0,Jack Nicholson,359.0,Actor,642.0,8,9.982966e+07
13582,The Departed,2006,8.5,85.0,1132417,151,R,Drama,Martin Scorsese,Jack Nicholson,...,2006-10-06,90000000.0,132384315.0,289660619.0,Jack Nicholson,359.0,Actor,642.0,8,9.982966e+07
13581,The Departed,2006,8.5,85.0,1132417,151,R,Crime,Martin Scorsese,Jack Nicholson,...,2006-10-06,90000000.0,132384315.0,289660619.0,Jack Nicholson,359.0,Actor,642.0,8,9.982966e+07
9927,Something's Gotta Give,2003,6.7,66.0,107716,128,PG-13,Romance,Nancy Meyers,Jack Nicholson,...,2003-12-12,80000000.0,124685242.0,266685242.0,Jack Nicholson,359.0,Actor,642.0,7,9.627887e+07
9926,Something's Gotta Give,2003,6.7,66.0,107716,128,PG-13,Drama,Nancy Meyers,Jack Nicholson,...,2003-12-12,80000000.0,124685242.0,266685242.0,Jack Nicholson,359.0,Actor,642.0,7,9.627887e+07
9925,Something's Gotta Give,2003,6.7,66.0,107716,128,PG-13,Comedy,Nancy Meyers,Jack Nicholson,...,2003-12-12,80000000.0,124685242.0,266685242.0,Jack Nicholson,359.0,Actor,642.0,7,9.627887e+07
10091,Anger Management,2003,6.3,52.0,197714,106,PG-13,Comedy,Peter Segal,Jack Nicholson,...,2003-04-11,56000000.0,135560942.0,195660942.0,Jack Nicholson,359.0,Actor,642.0,6,8.973185e+07
8861,About Schmidt,2002,7.1,85.0,119821,125,R,Comedy,Alexander Payne,Jack Nicholson,...,2002-12-13,30000000.0,65005217.0,107054484.0,Jack Nicholson,359.0,Actor,642.0,5,9.467718e+07


# Create a new feature called 'Director Power', which will be measured as the average domestic gross of the director's previous movies:

In [1778]:
merged_df.head()

,movie,year,imdb,metascore,votes,runtime,certificate,genre,director,stars,...,release_date,production_budget,domestic_gross,worldwide_gross,star_name,star_ranking,actor_or_actress,star_points,star_appearances,star_power
0,Major League,1989,7.2,62.0,59421,107,R,Comedy,David S. Ward,Tom Berenger,...,1989-04-07,11000000.0,49793054.0,49793054.0,NaN,NaN,NaN,0.0,0,0.0
1,Major League,1989,7.2,62.0,59421,107,R,Sport,David S. Ward,Tom Berenger,...,1989-04-07,11000000.0,49793054.0,49793054.0,NaN,NaN,NaN,0.0,0,0.0
2,Major League,1989,7.2,62.0,59421,107,R,Comedy,David S. Ward,Charlie Sheen,...,1989-04-07,11000000.0,49793054.0,49793054.0,NaN,NaN,NaN,0.0,0,0.0
3,Major League,1989,7.2,62.0,59421,107,R,Sport,David S. Ward,Charlie Sheen,...,1989-04-07,11000000.0,49793054.0,49793054.0,NaN,NaN,NaN,0.0,0,0.0
4,Major League,1989,7.2,62.0,59421,107,R,Comedy,David S. Ward,Corbin Bernsen,...,1989-04-07,11000000.0,49793054.0,49793054.0,NaN,NaN,NaN,0.0,0,0.0


In [1779]:
merged_df[merged_df.director=='Christopher Nolan']

,movie,year,imdb,metascore,votes,runtime,certificate,genre,director,stars,...,release_date,production_budget,domestic_gross,worldwide_gross,star_name,star_ranking,actor_or_actress,star_points,star_appearances,star_power
8356,Insomnia,2002,7.2,78.0,262794,118,R,Drama,Christopher Nolan,Al Pacino,...,2002-05-24,46000000.0,67263182.0,113622499.0,Al Pacino,188.0,Actor,813.0,7,5.514386e+07
8357,Insomnia,2002,7.2,78.0,262794,118,R,Mystery,Christopher Nolan,Al Pacino,...,2002-05-24,46000000.0,67263182.0,113622499.0,Al Pacino,188.0,Actor,813.0,7,5.514386e+07
8358,Insomnia,2002,7.2,78.0,262794,118,R,Thriller,Christopher Nolan,Al Pacino,...,2002-05-24,46000000.0,67263182.0,113622499.0,Al Pacino,188.0,Actor,813.0,7,5.514386e+07
8359,Insomnia,2002,7.2,78.0,262794,118,R,Drama,Christopher Nolan,Robin Williams,...,2002-05-24,46000000.0,67263182.0,113622499.0,Robin Williams,341.0,Actor,660.0,14,9.617102e+07
8360,Insomnia,2002,7.2,78.0,262794,118,R,Mystery,Christopher Nolan,Robin Williams,...,2002-05-24,46000000.0,67263182.0,113622499.0,Robin Williams,341.0,Actor,660.0,14,9.617102e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30155,Dunkirk,2017,7.9,94.0,509863,106,PG-13,Drama,Christopher Nolan,Mark Rylance,...,2017-07-18,150000000.0,190068280.0,527268280.0,NaN,NaN,NaN,0.0,2,6.389876e+07
30156,Dunkirk,2017,7.9,94.0,509863,106,PG-13,History,Christopher Nolan,Mark Rylance,...,2017-07-18,150000000.0,190068280.0,527268280.0,NaN,NaN,NaN,0.0,2,6.389876e+07
30157,Dunkirk,2017,7.9,94.0,509863,106,PG-13,Action,Christopher Nolan,Tom Hardy,...,2017-07-18,150000000.0,190068280.0,527268280.0,Tom Hardy,39.0,Actor,962.0,10,9.291996e+07
30158,Dunkirk,2017,7.9,94.0,509863,106,PG-13,Drama,Christopher Nolan,Tom Hardy,...,2017-07-18,150000000.0,190068280.0,527268280.0,Tom Hardy,39.0,Actor,962.0,10,9.291996e+07


In [1780]:
merged_df = merged_df.sort_values(by='release_date', ascending=True)

for each movie, check director's past movies (prior release dates) and take the avg of domestic gross:

In [1781]:
merged_df['director_power'] = merged_df.apply(
    lambda x: merged_df[(merged_df.director == x.director) & (merged_df.release_date < x.release_date)].domestic_gross.mean(), axis=1)

In [1787]:
merged_df[merged_df.director=='Christopher Nolan'].sort_values(by='release_date', ascending=False)

,movie,year,imdb,metascore,votes,runtime,certificate,genre,director,stars,...,production_budget,domestic_gross,worldwide_gross,star_name,star_ranking,actor_or_actress,star_points,star_appearances,star_power,director_power
30159,Dunkirk,2017,7.9,94.0,509863,106,PG-13,History,Christopher Nolan,Tom Hardy,...,150000000.0,190068280.0,527268280.0,Tom Hardy,39.0,Actor,962.0,10,9.291996e+07,2.483435e+08
30157,Dunkirk,2017,7.9,94.0,509863,106,PG-13,Action,Christopher Nolan,Tom Hardy,...,150000000.0,190068280.0,527268280.0,Tom Hardy,39.0,Actor,962.0,10,9.291996e+07,2.483435e+08
30153,Dunkirk,2017,7.9,94.0,509863,106,PG-13,History,Christopher Nolan,Barry Keoghan,...,150000000.0,190068280.0,527268280.0,NaN,NaN,NaN,0.0,0,0.000000e+00,2.483435e+08
30148,Dunkirk,2017,7.9,94.0,509863,106,PG-13,Action,Christopher Nolan,Fionn Whitehead,...,150000000.0,190068280.0,527268280.0,NaN,NaN,NaN,0.0,0,0.000000e+00,2.483435e+08
30154,Dunkirk,2017,7.9,94.0,509863,106,PG-13,Action,Christopher Nolan,Mark Rylance,...,150000000.0,190068280.0,527268280.0,NaN,NaN,NaN,0.0,2,6.389876e+07,2.483435e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,Insomnia,2002,7.2,78.0,262794,118,R,Mystery,Christopher Nolan,Al Pacino,...,46000000.0,67263182.0,113622499.0,Al Pacino,188.0,Actor,813.0,7,5.514386e+07,0.000000e+00
8359,Insomnia,2002,7.2,78.0,262794,118,R,Drama,Christopher Nolan,Robin Williams,...,46000000.0,67263182.0,113622499.0,Robin Williams,341.0,Actor,660.0,14,9.617102e+07,0.000000e+00
8358,Insomnia,2002,7.2,78.0,262794,118,R,Thriller,Christopher Nolan,Al Pacino,...,46000000.0,67263182.0,113622499.0,Al Pacino,188.0,Actor,813.0,7,5.514386e+07,0.000000e+00
8362,Insomnia,2002,7.2,78.0,262794,118,R,Drama,Christopher Nolan,Hilary Swank,...,46000000.0,67263182.0,113622499.0,Hilary Swank,495.0,Actress,506.0,1,1.154061e+07,0.000000e+00


Fill NA with zeroes:

In [1783]:
merged_df['director_power'].fillna(0, inplace=True)

In [1784]:
merged_df.sample(5)

,movie,year,imdb,metascore,votes,runtime,certificate,genre,director,stars,...,production_budget,domestic_gross,worldwide_gross,star_name,star_ranking,actor_or_actress,star_points,star_appearances,star_power,director_power
21825,Colombiana,2011,6.4,45.0,90796,108,PG-13,Drama,Olivier Megaton,Zoe Saldana,...,40000000.0,36665854.0,72295262.0,Zoe Saldana,202.0,Actress,799.0,5,189320354.8,31715062.0
23315,Act of Valor,2012,6.5,40.0,64828,110,R,Action,Mike McCoy,Roselyn Sanchez,...,12000000.0,70012847.0,82497035.0,Roselyn Sanchez,96.0,Actress,905.0,1,90648202.0,0.0
1810,Species,1995,5.8,49.0,71650,108,R,Sci-Fi,Roger Donaldson,Alfred Molina,...,35000000.0,60054449.0,113354449.0,NaN,NaN,NaN,0.0,0,0.0,0.0
22737,Argo,2012,7.7,86.0,554267,120,R,Drama,Ben Affleck,Alan Arkin,...,44500000.0,136025503.0,227140757.0,NaN,NaN,NaN,0.0,4,39070813.5,92186262.0
6378,Big Momma's House,2000,5.2,33.0,78608,99,PG-13,Crime,Raja Gosnell,Paul Giamatti,...,33000000.0,117559438.0,173559438.0,NaN,NaN,NaN,0.0,0,0.0,0.0


# Additional cleaning and features:

Convert production budget, domestic gross and worldwide gross to int:

In [1788]:
merged_df.production_budget = merged_df.production_budget.astype(int)
merged_df.domestic_gross = merged_df.domestic_gross.astype(int)
merged_df.worldwide_gross = merged_df.worldwide_gross.astype(int)

Add title length and release month as additional features:

In [1789]:
merged_df['title_length'] = merged_df.movie.str.len() #number of characters in movie title added as new feature

In [1790]:
merged_df['month'] = merged_df['release_date'].dt.strftime('%b') #month of movie release

In [1791]:
merged_df.head(3)

,movie,year,imdb,metascore,votes,runtime,certificate,genre,director,stars,...,worldwide_gross,star_name,star_ranking,actor_or_actress,star_points,star_appearances,star_power,director_power,title_length,month
92,Bill & Ted's Excellent Adventure,1989,6.9,44.0,105144,90,PG,Adventure,Stephen Herek,Keanu Reeves,...,40485039,Keanu Reeves,115.0,Actor,886.0,0,0.0,0.0,32,Feb
93,Bill & Ted's Excellent Adventure,1989,6.9,44.0,105144,90,PG,Comedy,Stephen Herek,Keanu Reeves,...,40485039,Keanu Reeves,115.0,Actor,886.0,0,0.0,0.0,32,Feb
94,Bill & Ted's Excellent Adventure,1989,6.9,44.0,105144,90,PG,Music,Stephen Herek,Keanu Reeves,...,40485039,Keanu Reeves,115.0,Actor,886.0,0,0.0,0.0,32,Feb


Pickle merged (expanded) dataframe:

In [1793]:
merged_df.to_pickle('merged_df.pkl')

In [1792]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33708 entries, 92 to 32620
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   movie              33708 non-null  object        
 1   year               33708 non-null  int64         
 2   imdb               33708 non-null  float64       
 3   metascore          33708 non-null  float64       
 4   votes              33708 non-null  int64         
 5   runtime            33708 non-null  int64         
 6   certificate        33708 non-null  object        
 7   genre              33708 non-null  object        
 8   director           33708 non-null  object        
 9   stars              33708 non-null  object        
 10  genre_count        33708 non-null  int64         
 11  release_date       33708 non-null  datetime64[ns]
 12  production_budget  33708 non-null  int64         
 13  domestic_gross     33708 non-null  int64         
 14  world

# Aggregate the stars and genre data

Aggregate the stars data, by averaging the "star power" and "star points", and summing the "star appearances" for each movie:

In [1907]:
groupby_cols = ['movie', 'year', 'imdb', 'metascore', 'votes', 'runtime', 'title_length',
                'certificate', 'genre', 'genre_count', 'director', 'release_date', 'month',
                'production_budget', 'domestic_gross', 'worldwide_gross',
                'director_power']

movies_genre_df = merged_df.groupby(groupby_cols).agg({'star_power':'mean', 'star_points':'mean', 'star_appearances':'sum'}).reset_index()

In [1908]:
movies_genre_df.head(3)

,movie,year,imdb,metascore,votes,runtime,title_length,certificate,genre,genre_count,director,release_date,month,production_budget,domestic_gross,worldwide_gross,director_power,star_power,star_points,star_appearances
0,10 Cloverfield Lane,2016,7.2,76.0,276310,103,19,PG-13,Drama,3,Dan Trachtenberg,2016-01-04,Jan,15000000,72082999,108286422,0.0,2.793540e+07,240.0,24
1,10 Cloverfield Lane,2016,7.2,76.0,276310,103,19,PG-13,Horror,3,Dan Trachtenberg,2016-01-04,Jan,15000000,72082999,108286422,0.0,2.793540e+07,240.0,24
2,10 Cloverfield Lane,2016,7.2,76.0,276310,103,19,PG-13,Mystery,3,Dan Trachtenberg,2016-01-04,Jan,15000000,72082999,108286422,0.0,2.793540e+07,240.0,24


In [1909]:
movies_genre_df.columns

Index(['movie', 'year', 'imdb', 'metascore', 'votes', 'runtime',
       'title_length', 'certificate', 'genre', 'genre_count', 'director',
       'release_date', 'month', 'production_budget', 'domestic_gross',
       'worldwide_gross', 'director_power', 'star_power', 'star_points',
       'star_appearances'],
      dtype='object')

In [1910]:
movies_genre_df.shape

(8256, 20)

Rearrange columns:

In [1911]:
movies_genre_df = movies_genre_df[['movie', 'imdb', 'metascore', 'votes', 'runtime', 'certificate',
                       'year', 'month', 'release_date', 'genre', 'genre_count',
                       'director', 'title_length', 'production_budget',   
                       'director_power', 'star_power', 'star_appearances', 'star_points', 
                       'worldwide_gross', 'domestic_gross']] 

Now create another dataframe collapsing on genre, where now we'll have one row per movie:

In [1930]:
groupby_cols = ['movie', 'year', 'imdb', 'metascore', 'votes', 'runtime', 'title_length',
                'certificate', 'genre_count', 'director', 'release_date', 'month',
                'production_budget', 'domestic_gross', 'worldwide_gross',
                'director_power', 'star_power', 'star_appearances', 'star_points']

movies_df = movies_genre_df[groupby_cols].drop_duplicates(subset=['movie', 'release_date'])
movies_df.reset_index(drop=True, inplace=True)

In [1935]:
movies_df.head(3)

,movie,year,imdb,metascore,votes,runtime,title_length,certificate,genre_count,director,release_date,month,production_budget,domestic_gross,worldwide_gross,director_power,star_power,star_appearances,star_points
0,10 Cloverfield Lane,2016,7.2,76.0,276310,103,19,PG-13,3,Dan Trachtenberg,2016-01-04,Jan,15000000,72082999,108286422,0.0,2.793540e+07,24,240.00
1,10 Things I Hate About You,1999,7.3,70.0,282841,97,26,PG-13,3,Gil Junger,1999-03-31,Mar,13000000,38177966,60413950,0.0,0.000000e+00,0,628.75
2,102 Dalmatians,2000,4.9,35.0,33368,100,14,G,3,Kevin Lima,2000-11-22,Nov,85000000,66941559,66941559,0.0,5.724807e+07,3,0.00


In [1936]:
movies_df.shape

(3148, 19)

In [1937]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3148 entries, 0 to 3147
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   movie              3148 non-null   object        
 1   year               3148 non-null   int64         
 2   imdb               3148 non-null   float64       
 3   metascore          3148 non-null   float64       
 4   votes              3148 non-null   int64         
 5   runtime            3148 non-null   int64         
 6   title_length       3148 non-null   int64         
 7   certificate        3148 non-null   object        
 8   genre_count        3148 non-null   int64         
 9   director           3148 non-null   object        
 10  release_date       3148 non-null   datetime64[ns]
 11  month              3148 non-null   object        
 12  production_budget  3148 non-null   int64         
 13  domestic_gross     3148 non-null   int64         
 14  worldwid

In [1938]:
movies_df.columns

Index(['movie', 'year', 'imdb', 'metascore', 'votes', 'runtime',
       'title_length', 'certificate', 'genre_count', 'director',
       'release_date', 'month', 'production_budget', 'domestic_gross',
       'worldwide_gross', 'director_power', 'star_power', 'star_appearances',
       'star_points'],
      dtype='object')

In [1940]:
movies_df = movies_df[['movie', 'imdb', 'metascore', 'votes', 'runtime', 'certificate',
                       'year', 'month', 'release_date', 'genre_count',
                       'director', 'title_length', 'production_budget',   
                       'director_power', 'star_power', 'star_appearances', 'star_points',
                       'worldwide_gross', 'domestic_gross']] 

Pickle both the movies and movies-genre dataframe:

In [1941]:
movies_df.to_pickle('movies_df.pkl')
movies_genre_df.to_pickle('movies_genre_df.pkl')

In [1942]:
movies_df.to_csv('movies_df.csv')
movies_genre_df.to_csv('movies_genre_df.csv')

# We can now use these two dataframes for analysis

# Transform for modeling -REMOVE FROM THIS NB

One-hot encode the categorical features including genre, certificate and release month:

In [1690]:
movies_matrix = pd.get_dummies(movies_df, prefix=['genre'], columns=['genre'])

In [1693]:
movies_matrix = pd.get_dummies(movies_matrix, prefix=['certificate', 'month'], columns=['certificate', 'month'], drop_first=True)

In [1688]:
movies_matrix.head(3)

,movie,imdb,metascore,votes,runtime,year,release_date,genre,genre_count,director,...,month_Dec,month_Feb,month_Jan,month_Jul,month_Jun,month_Mar,month_May,month_Nov,month_Oct,month_Sep
0,10 Cloverfield Lane,7.2,76.0,276310,103,2016,2016-01-04,Drama,3,Dan Trachtenberg,...,0,0,1,0,0,0,0,0,0,0
1,10 Cloverfield Lane,7.2,76.0,276310,103,2016,2016-01-04,Horror,3,Dan Trachtenberg,...,0,0,1,0,0,0,0,0,0,0
2,10 Cloverfield Lane,7.2,76.0,276310,103,2016,2016-01-04,Mystery,3,Dan Trachtenberg,...,0,0,1,0,0,0,0,0,0,0


In [1694]:
movies_matrix.columns

Index(['movie', 'imdb', 'metascore', 'votes', 'runtime', 'year',
       'release_date', 'genre_count', 'director', 'title_length',
       'production_budget', 'director_power', 'star_points', 'star_power',
       'worldwide_gross', 'domestic_gross', 'genre_Action', 'genre_Adventure',
       'genre_Animation', 'genre_Biography', 'genre_Comedy', 'genre_Crime',
       'genre_Documentary', 'genre_Drama', 'genre_Family', 'genre_Fantasy',
       'genre_History', 'genre_Horror', 'genre_Music', 'genre_Musical',
       'genre_Mystery', 'genre_News', 'genre_Romance', 'genre_Sci-Fi',
       'genre_Sport', 'genre_Thriller', 'genre_War', 'genre_Western',
       'certificate_NC-17', 'certificate_Not Rated', 'certificate_PG',
       'certificate_PG-13', 'certificate_R', 'month_Aug', 'month_Dec',
       'month_Feb', 'month_Jan', 'month_Jul', 'month_Jun', 'month_Mar',
       'month_May', 'month_Nov', 'month_Oct', 'month_Sep'],
      dtype='object')

Aggregate the dataframe to one row per movie by summing the encoded genre features:

In [1697]:
groupby_cols = ['movie', 'imdb', 'metascore', 'votes', 'runtime', 'year',
               'release_date', 'genre_count', 'director', 'title_length',
               'production_budget', 'director_power', 'star_points', 'star_power',
               'worldwide_gross', 'domestic_gross',
               'certificate_NC-17', 'certificate_Not Rated', #'certificate_G' 
               'certificate_PG', 'certificate_PG-13', 'certificate_R', 
               'month_Sep', 'month_Jul', 'month_Oct', #'month_Apr'
               'month_Aug', 'month_Dec', 'month_Feb', 'month_Jan', 
               'month_Jun', 'month_Mar', 'month_May', 'month_Nov']

encoded_cols = ['genre_Action', 'genre_Adventure',
               'genre_Animation', 'genre_Biography', 'genre_Comedy', 'genre_Crime',
               'genre_Documentary', 'genre_Drama', 'genre_Family', 'genre_Fantasy',
               'genre_History', 'genre_Horror', 'genre_Music', 'genre_Musical',
               'genre_Mystery', 'genre_News', 'genre_Romance', 'genre_Sci-Fi',
               'genre_Sport', 'genre_Thriller', 'genre_War', 'genre_Western']

movies_matrix = movies_matrix.groupby(groupby_cols, as_index=False)[encoded_cols].sum()

In [1698]:
movies_matrix.head(3)

,movie,imdb,metascore,votes,runtime,year,release_date,genre_count,director,title_length,...,genre_Music,genre_Musical,genre_Mystery,genre_News,genre_Romance,genre_Sci-Fi,genre_Sport,genre_Thriller,genre_War,genre_Western
0,10 Cloverfield Lane,7.2,76.0,276310,103,2016,2016-01-04,3,Dan Trachtenberg,19,...,0,0,1,0,0,0,0,0,0,0
1,10 Things I Hate About You,7.3,70.0,282841,97,1999,1999-03-31,3,Gil Junger,26,...,0,0,0,0,1,0,0,0,0,0
2,102 Dalmatians,4.9,35.0,33368,100,2000,2000-11-22,3,Kevin Lima,14,...,0,0,0,0,0,0,0,0,0,0


Save movies matrix to pickle and csv:

In [1699]:
movies_matrix.to_pickle('movies_matrix.pkl')

In [1700]:
movies_matrix.to_csv('movies_matrix.csv')

In [1701]:
movies_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3148 entries, 0 to 3147
Data columns (total 54 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   movie                  3148 non-null   object        
 1   imdb                   3148 non-null   float64       
 2   metascore              3148 non-null   float64       
 3   votes                  3148 non-null   int64         
 4   runtime                3148 non-null   int64         
 5   year                   3148 non-null   int64         
 6   release_date           3148 non-null   datetime64[ns]
 7   genre_count            3148 non-null   int64         
 8   director               3148 non-null   object        
 9   title_length           3148 non-null   int64         
 10  production_budget      3148 non-null   int64         
 11  director_power         3148 non-null   float64       
 12  star_points            3148 non-null   float64       
 13  sta